Скрепингом извлекали данные с сайта https://rus.hitmotop.com 

Сделала настройки в коде (options) для того, чтобы можно было разделить весь процесс и запустить параллельно несколько отрезков 

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
import urllib.parse

def read_artists_from_csv(filename):
    artists = []
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if 'Rank' in row and row['Rank']:
                    artists.append(row['Rank'])
        print(f"Прочитано артистов из CSV: {len(artists)}")
        return artists
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
        return []

def search_artist(artist_name):
    search_url = f"https://rus.hitmotop.com/search?q={urllib.parse.quote(artist_name)}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        print(f"Ищем артиста: {artist_name}")
        response = requests.get(search_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        #шарим по странице для раздела Исполнители
        performers_section = None
        for h2 in soup.find_all('h2'):
            if 'Исполнители' in h2.get_text():
                performers_section = h2.find_next('ul')
                break
        
        if performers_section:
            #начинаем с первого артиста
            first_artist_link = performers_section.find('a', href=lambda x: x and x.startswith('/artist/'))
            if first_artist_link:
                artist_href = first_artist_link.get('href')
                artist_id = artist_href.split('/artist/')[-1]
                print(f"Найден артист: {artist_name} -> id: {artist_id}")
                return artist_id
        
        print(f"Артист не найден: {artist_name}")
        return None
        
    except Exception as e:
        print(f"Ошибка поиска артиста {artist_name}: {e}")
        return None

def get_artist_tracks(artist_id, artist_name, max_pages=1):
    base_url = f"https://rus.hitmotop.com/artist/{artist_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    all_tracks = []
    
    for page in range(1, max_pages + 1):
        try:
            if page == 1:
                url = base_url
            else:
                start_num = (page - 1) * 48
                url = f"{base_url}/start/{start_num}"
            
            print(f"Страница {page}: {url}")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            #поиск ссылок на трэки
            track_elements = soup.select('a[href^="/song/"]')
            
            for track_element in track_elements:
                href = track_element.get('href')
                if href and href.startswith('/song/'):
                    track_id = href.split('/song/')[-1]
                    
                    #каждый трэк
                    track_info = get_track_info(track_id, artist_name)
                    if track_info:
                        track_info['artist_id'] = artist_id
                        all_tracks.append(track_info)
            
            print(f"Страница {page}: найдено {len(track_elements)} треков")
            
            #выходим, если нет следующей страницы
            if not soup.select('a[href*="/start/"]'):
                break
                
            time.sleep(random.uniform(1, 2))
            
        except Exception as e:
            print(f"ошибка на странице {page}: {e}")
            continue
    
    return all_tracks

def get_track_info(track_id, artist_name):
    track_url = f"https://rus.hitmotop.com/song/{track_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(track_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        #title
        title_element = soup.find('h1', class_='p-track-title')
        if not title_element:
            return None
        
        track_text = title_element.get_text(strip=True)
        
        #title without artist
        if ' - ' in track_text:
            title = track_text.split(' - ', 1)[1]
        else:
            title = track_text
        
        #time
        duration_element = soup.select_one('.track__fulltime')
        duration = duration_element.get_text(strip=True) if duration_element else 'N/A'
        
        track_info = {
            'artist': artist_name,
            'artist_id': '',
            'track_id': track_id,
            'title': title,
            'duration': duration,
            'url': track_url
        }
        
        print(f"{artist_name} - {title} | {duration}")
        return track_info
        
    except Exception as e:
        print(f"Ошибка получения информации о треке {track_id}: {e}")
        return None

#сливаем в файл
def save_tracks_to_csv(tracks, filename):
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
            writer = csv.writer(file)
            writer.writerow(['Исполнитель', 'ID исполнителя', 'ID песни', 'Название песни', 'Продолжительность', 'Ссылка'])
            
            for track in tracks:
                writer.writerow([
                    track['artist'],
                    track['artist_id'],
                    track['track_id'],
                    track['title'],
                    track['duration'],
                    track['url']
                ])
        
        print(f"Сохранено {len(tracks)} треков в файл: {filename}")
        return True
        
    except Exception as e:
        print(f"Ошибка сохранения CSV: {e}")
        return False

def main():
    print("PARSER")
    
    csv_filename = 'spotify_artists.csv'
    artists = read_artists_from_csv(csv_filename)
    
    if not artists:
        print("Не удалось прочитать артистов из CSV файла")
        return
    
    #options!!!!! важно
    start_index = 0      #начальный индекс артиста (0 = первый)
    end_index = 3000      #конечный индекс артиста (1 = только первый)
    max_pages_per_artist = 1  #макс количество страниц на артиста
    
    print(f"\n Парсим артистов с {start_index} по {end_index} индекс")
    
    all_tracks = []
    
    for i in range(start_index, min(end_index, len(artists))):
        artist_name = artists[i]
        print(f"\n{'='*50}")
        print(f"Обрабатываем артиста {i+1}/{min(end_index, len(artists))}: {artist_name}")
        
        #ищем id
        artist_id = search_artist(artist_name)
        if not artist_id:
            continue
        
        #+ трэки
        artist_tracks = get_artist_tracks(artist_id, artist_name, max_pages_per_artist)
        all_tracks.extend(artist_tracks)
        
        print(f"Артист {artist_name}: собрано {len(artist_tracks)} треков")
        
        time.sleep(random.uniform(2, 4))

    if all_tracks:
        output_filename = 'artist_tracks2950_3000.csv'
        save_tracks_to_csv(all_tracks, output_filename)
        print(f"\n собрали трэков: {len(all_tracks)}")
        print(f" Файл: {output_filename}")
    else:
        print("\n Не удалось собрать ни одного трека")

if __name__ == "__main__":
    main()
    print("\n нажмите enter для выхода...")
    input()

Данный код предназначен для парсинга информации о треках исполнителей с сайта Hitmotop. Код последовательно обрабатывает артистов из файла spotify_artists.csv, беря имена из столбца Rank.

Алгоритм работы:

1. Поиск артиста осуществляется через адрес https://rus.hitmotop.com/search?q={имя_артиста}
2. В разделе с заголовком "Исполнители" извлекается первая ссылка вида /artist/{номер}, которая является идентификатором артиста
3. Для каждого артиста парсятся треки с его страницы. Из основного контента извлекаются идентификаторы песен в формате /song/{номер}
4. Для каждого трека переходим на его страницу и извлекаем:
- Название трека из элемента h1 с классом p-track-title
- Длительность трека из элемента с классом track__fulltime
5. Для пагинации используется адрес вида /artist/{номер}/start/{смещение}, где смещение увеличивается на 48 для каждой следующей страницы
6. Все собранные данные сохраняются в CSV файл с колонками: Исполнитель, ID исполнителя, ID песни, Название песни, Продолжительность трека

Код позволяет настраивать диапазон обрабатываемых артистов через параметры start_index и end_index. Для корректного отображения кириллических символов используется кодировка utf-8-sig.

Информация о структуре HTML и способах извлечения данных была получена с помощью инструмента DevTools из гипертекстовой разметки сайта.

Работа производилась отрезками, после был использован код для того, чтобы слить полученные файлы, вот код:

In [ ]:
import pandas as pd

files = [
    "artist_tracks0_100.csv",
    "artist_tracks101.csv",
    "artist_tracks101_200.csv",
    "artist_tracks200_201.csv",
    "artist_tracks201_400.csv",
    "artist_tracks400_500.csv",
    "artist_tracks500_600.csv",
    "artist_tracks600_700.csv",
    "artist_tracks700_800.csv",
    "artist_tracks801-1000.csv",
    "artist_tracks1001_2000.csv",
    "artist_tracks2000_2100.csv",
    "artist_tracks2100_2200.csv",
    "artist_tracks2200_2250.csv",
    "artist_tracks2250_2300.csv",
    "artist_tracks2300_2350.csv",
    "artist_tracks2350_2400.csv",
    "artist_tracks2400_2500.csv",
    "artist_tracks2500_2600.csv",
    "artist_tracks2600_2700.csv",
    "artist_tracks2700_2800.csv",
    "artist_tracks2800_2850.csv",
    "artist_tracks2850_2900.csv",
    "artist_tracks2900_2950.csv",
    "artist_tracks2950_3000.csv"
]

combined_df = pd.DataFrame()

for file in files:
    try:
        df = pd.read_csv(file)
        
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        
        print(f"Успех: {file}")
        
    except FileNotFoundError:
        print(f"нет айла: {file}")
    except Exception as e:
        print(f"ошибка в чтении{file}: {e}")

output_file = "hitmotop_artist_tracks.csv"
combined_df.to_csv(output_file, index=False)

print(f"\n итоговый файл: {output_file}")

Код должен находиться в одной папке с файлами.